In [ ]:
files = list(input.files)
assemblies = params.assemblies
thresh = params.mapq_thresh

# get the color (v important)
sample_1 = wildcards.sample.split('_')[0]
meta = load_meta()
pop = meta.loc[meta['sample'] == sample_1, 'population'].values[0]
c_dict, _ = get_population_colors()
color = c_dict[pop]

# get the upset plot table
i = 0
for f, a in zip(files, assemblies):
    temp = pd.read_csv(f, sep='\t')
    temp.rename({'mapq':a}, axis=1, inplace=True)
    assert len(temp.index) == len(temp.read_id.unique())

    if i == 0:
        df = temp.copy(deep=True)
    else:
        df = df.merge(temp, how='outer', on='read_id')
    i += 1

# convert to binary
df.fillna(0, inplace=True)
df.set_index('read_id', inplace=True)
df = df>thresh

df.reset_index(inplace=True)
df.set_index(assemblies, inplace=True)

# make the upset plot
ax_dict = upsetplot.UpSet(df, subset_size='count',
                  facecolor=color,
                  sort_by='cardinality',
                  show_counts=False,
                  show_percentages=True).plot()
plt.suptitle(f'% of {sample} reads w/ mapq>{thresh}')
plt.savefig(output.upset, dpi=500)

# get the afr only reads
afr_reads = df.copy(deep=True)
afr_reads.reset_index(inplace=True)

non_afr_assemblies = list(set(assemblies)-set(['afr']))

afr_reads = afr_reads.loc[(afr_reads.afr==True)]
for a in non_afr_assemblies:
    afr_reads = afr_reads.loc[afr_reads[a]==False]
afr_reads = afr_reads[['read_id']]
afr_reads.to_csv(output.afr_reads, index=False)

# get the summary table
df['total_reads'] = df.n_reads.sum()
df['perc'] = (df.n_reads/df.total_reads)*100
df['sample'] = sample
df.to_csv(output.tsv, sep='\t', index=False)